In [1]:
import torch
from torch import nn
from torch.nn import functional as F # 里面定义了一些无参的函数

In [2]:
# Sequential是一种特殊的Module. 他也继承nn.Module. 一会我们会看到它是如何被实现的.
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) # 创建一多层感知机模型
# Sequential是一个容器. 主要
# 我们通过sequential定义了一个块, 里面的每一个Module是一个层. 
X = torch.rand(2, 20) # 随机生成模拟数据
out = net(X)
X.shape, out.shape

(torch.Size([2, 20]), torch.Size([2, 10]))

In [7]:
# 自定义块:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # hidden layer
        self.out = nn.Linear(256, 10) # output layer 
    
    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))
        

In [9]:
# 使用我们创建的块, 创建网络模型
net = MLP() # 实例化

out = net(X) # 实例调用__call_方法, 直接调用forward

out.shape, X.shape # x经过net得到了输出.

(torch.Size([2, 10]), torch.Size([2, 20]))

In [6]:
# 顺序块: 现在我们可以更加仔细地看看Sequential类是如何工作的.
# 我们利用Module来实现一下: 
# from torch import Module


# 自定义的Sequential:
class MySequential(nn.Module):
    def __init__(self, *args,) -> None:
        super().__init__(*args,)
        for idx, module in enumerate(args): # args is a tuple. *args are scattered data item. 
            self._modules[str(idx)] = module
            # _modules是torch.nn.Module中的一个特殊变量, 是一个orderedDict()即有序字典.
            
    def forward(self, x):
        for module in self._modules.values():
            x = module(x)
        return x

 
# class MySequential(nn.Module):
#     def __init__(self, *args) -> None:
#         super().__init__()
#         for idx, module in enumerate(args):
#             self._modules[str(idx)] = module # __modules是torch中的一个特殊变量
#             # 是一个迭代器, orderedDict()
#         # for block in args:
#             # self._modules[block] = block # 这是一种特殊的

#     def forward(self, X):
#          # OrderDict保证了按照成员添加的顺序遍历他们.
#         for block in self._modules.values(): # 取出
#             X = block(X)
        
#         return X
    

In [7]:
# 几乎和我们使用Sequential是一样的, 我们传入的也是层参数, 然后返回一个实例
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

out = net(X)

out.shape, X.shape

(torch.Size([2, 10]), torch.Size([2, 20]))

In [8]:
# 有了Sequential我们为什么还要使用Module. 这种显示创建的形式.
# 因为我们可以在init和forward中做一些代码操作. 这样使我们的网络模型更加灵活
class FixedHiddenMLP(nn.Module): # 固定隐藏层
    def __init__(self) -> None:
        super().__init__()
        # 不计算其中的权重梯度. 那么更新时在训练期间保持不变.
        self.rand_weight = torch.rand(size=(20, 20), requires_grad=False) # 不计算梯度, 也就是固定住了.
        # 因为不更新, 所以是随机加上了一层参数.
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用functional中的relu函数:
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # mm: matmul就是矩阵乘积
        # 复用Linear层, 这表示两个层之间共享参数. 
        # 当我们想要将两个层的参数共享的时候, 我们就是用这种方式w
        X = self.linear(X)

        # 控制流:
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
        

In [11]:
# 测试, 在这个FixedHiddenMLP Module中, 权重: rand_weight永远不会被更新, 从初始化后就一直维持常量的状态.
net = FixedHiddenMLP() # 实例
out = net(X)

out, out.shape, X.shape

(tensor(-0.0775, grad_fn=<SumBackward0>), torch.Size([]), torch.Size([2, 20]))

In [16]:
# 我们可以混合块和Sequential去使用.
# 因为实际上无论是我们自定义的块, 还是Sequential都是nn.Module的子类. 所以我们当然可以嵌套使用
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
# 可以看见, 我们首先经过NestMLP()的forward, 然后进入linear, 最后进入我们上面定义的FixdHiddenMLP层

out = chimera(X)
print(chimera) # 打印网络结构. 
out, out.shape, X.shape

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


(tensor(0.0981, grad_fn=<SumBackward0>), torch.Size([]), torch.Size([2, 20]))

## 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的顺序连接由`Sequential`块处理。

## 练习

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
1. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。
1. 假设你想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。


In [71]:
# 2:
class ParallelNet(nn.Module):
    def __init__(self, net1, net2) -> None:
        super().__init__()
        self.net1 = net1
        self.net2 = net2
    
    def forward(self, X):
        return self.net2(self.net1(X))
            
        


In [73]:
net = ParallelNet(
    nn.Linear(20, 20),
    nn.Linear(20, 10)
)



net(torch.rand(size=(2, 20))).shape

torch.Size([2, 10])